比较两种阈值的区别

In [1]:
import torch
th1 = torch.load('/home/lz/On-the-Fly_MoE_Inference/quantize/threshold/c4_mixtral_up/thresholds_0_8.pt')['up_proj_states_thresholds_2']
th_0 = torch.load('/home/lz/On-the-Fly_MoE_Inference/saving/threshold/c4_mixtral_up/thresholds_0_8.pt')['up_proj_states_thresholds_2']
for layerid in range(32):
    for expertid in range(8):
        counts = th1[layerid][expertid].numel()
        avg = th1[layerid][expertid].sum().item() / counts
        diff_0 = torch.abs(th1[layerid][expertid] - th_0[layerid][expertid]).sum() / counts
        print(f'layer {layerid}, expert {expertid}, 变化绝对值{diff_0:.4f} , 偏差比例 {(diff_0 / avg) * 100:.2f}% ')

### Train

In [1]:
import torch
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
from transformers import AutoTokenizer, BitsAndBytesConfig
from modeling_mixtral import MixtralForCausalLM, set_profile_mode
import json
from utils import get_model

# # 加载 C4 数据集的验证集
with open('../path.json', 'r') as file:
    paths = json.load(file)
    c4_path = paths.get('c4', '')
    model_name = paths.get('mixtral','')

with open('./device_map.json', 'r') as f:
    device_map = json.load(f)

set_profile_mode(False)
dtype = torch.bfloat16
print('using ',dtype)
llm, tokenizer = get_model(model_name, device_map, dtype=dtype)
# %%
#Quantize
from hqq.core.quantize import *
q4_config    = BaseQuantizeConfig(nbits=8, group_size=64) 
q3_config    = BaseQuantizeConfig(nbits=2, group_size=64)

quant_config = {
  'block_sparse_moe.experts.w3'  :q3_config,
}
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.quantize_model(llm, quant_config=quant_config, compute_dtype=dtype, device=device_map)


# print('quantize done')      

#First, quantize/load a quantized HQQ model the
from hqq.core.peft import PeftUtils
base_lora_params = {'lora_type':'default', 'r':128, 'lora_alpha':128, 'dropout':0.05, 'train_dtype':dtype}
up_lora_params = {'lora_type':'default', 'r':256, 'lora_alpha':256, 'dropout':0.05, 'train_dtype':dtype}

lora_params      = {'self_attn.q_proj': base_lora_params,
                    'self_attn.k_proj': base_lora_params,
                    'self_attn.v_proj': base_lora_params,
                    'self_attn.o_proj': base_lora_params,
                    'block_sparse_moe.experts.w1'   : base_lora_params,
                    'block_sparse_moe.experts.w3'   : up_lora_params,
                    'block_sparse_moe.experts.w2'   : base_lora_params}


#Add LoRA to linear/HQQ modules
PeftUtils.add_lora(llm, lora_params)
# PeftUtils.load_lora_weights(llm, model_save_path)
### replace eora to the LoRA in w3
for i in range(32):
    # print(f"Layer {i} done...")
    for j in range(8):
        # HQQLinearLoRA.state_dict()
        llmdevice = llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_A.device
        # print(llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_A.shape, llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_B.shape)
        a = torch.load('/home/lz/On-the-Fly_MoE_Inference/quantize/saved/eora/' + f'A_{i}_{j}.pt').to(dtype).to(llmdevice).transpose(0, 1)
        b = torch.load('/home/lz/On-the-Fly_MoE_Inference/quantize/saved/eora/' + f'B_prime_{i}_{j}.pt').to(dtype).to(llmdevice).transpose(0, 1)
        llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_A = torch.nn.Parameter(a)
        llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_B = torch.nn.Parameter(b)

        del a
        del b
        # print(llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_A.shape, llm.model.layers[i].block_sparse_moe.experts[j].w3.lora_B.shape)
        # llm.model.layers[i].block_sparse_moe.experts[j].w3 = \
        # )
        # CompensatedModel(llm.model.layers[i].block_sparse_moe.experts[j].w3, , layerid=i, expertid=j).to(llmdevice)

/home/lz/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lz/On-the-Fly_MoE_Inference/quantize/modeling_mixtral.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend y

Set profile_threshold to False
using  torch.bfloat16


100%|██████████| 929/929 [00:16<00:00, 56.95it/s]


### 加载c4和openmath数据集

In [3]:
from datasets import load_dataset, Dataset, concatenate_datasets
import functools

def preprocess_data(batch, tokenizer):
    # 使用 tokenizer 将文本数据转换为模型输入
    inputs = tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs["labels"] = inputs.input_ids.clone()
    return inputs


openmath = load_dataset("/home/lz/web-math/",data_files="/home/lz/web-math/openmath1.json")
c4 = load_dataset(c4_path)
# newdataset = openmath['train'] + c4['validation']
# print(openmath['train'], c4['validation'])
openmath_test = openmath['train']['text']  
c4_test = c4['validation']['text']  
# 如果数据是列表或字典形式，可以直接合并
combined_test = openmath_test + c4_test

# 如果需要将合并后的数据转换为 Hugging Face Dataset 对象
combined_dataset = Dataset.from_dict({"text": combined_test})

# openmath['train'][0]
test_num = 0.1
seed = 42

openmath_train = combined_dataset.train_test_split(test_size=test_num, seed=seed)
train_data = openmath_train['train']
test_data = openmath_train['test']

new_train_data = train_data.map(
    functools.partial(
    preprocess_data,
    tokenizer=tokenizer
), batched=True)
new_test_data = test_data.map(
    functools.partial(
    preprocess_data,
    tokenizer=tokenizer
), batched=True)


Repo card metadata block was not found. Setting CardData to empty.
2025-01-06:18:37:40,463 WARNING  [repocard.py:108] Repo card metadata block was not found. Setting CardData to empty.


In [8]:
new_train_data.shuffle(seed)
new_test_data.shuffle(seed)

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5558
})

In [9]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AdamW
from transformers import (
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)

model_save_path='./saved/training/'
learning_rate = 0.005
micro_batch_size=2
epochs=2
### don't save the model
save_steps = 1000000
save_total_limit = 2
sample_num = len(new_train_data)
optimizer=AdamW(filter(lambda p : p.requires_grad, llm.parameters()),lr=learning_rate)
linear_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(sample_num*epochs) // micro_batch_size)
args = TrainingArguments(
    output_dir=model_save_path,
    num_train_epochs=epochs,
    # max_steps=opt.max_steps,
    # fp16=True,
    bf16=True,
    optim="adamw_torch",# paged_adamw_8bit
    learning_rate=learning_rate,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=micro_batch_size,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,   ### 先设置成False
    group_by_length=False,
    logging_steps=100,
    eval_steps=100,
    save_strategy="steps",
    save_only_model=True,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    disable_tqdm=False,
)

trainer = Trainer(
    model=llm,
    train_dataset=new_train_data.select(range(3000)),
    eval_dataset=new_test_data.select(range(300)),
    args=args,
    optimizers=(optimizer, linear_scheduler),
    data_collator=DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True),
)

# silence the warnings. re-enable for inference!
llm.config.use_cache = False
trainer.train()

/home/lz/miniconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
2025-01-06:18:41:22,955 WARNING  [other.py:331] Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
100,7.225900
200,6.253300
300,6.383700
400,5.909900
500,5.897000
600,5.821400
700,5.709300
800,5.796700
900,5.911600
1000,5.702500


In [ ]:
#Convert LoRA weights to the same model dtype for faster inference
llm.eval()
PeftUtils.cast_lora_weights(llm, dtype=torch.bfloat16)

#Save LoRA weights
PeftUtils.save_lora_weights(llm, model_save_path+'lora_combine.pt')

# #Load LoRA weights: automatically calls add_lora 
# PeftUtils.load_lora_weights(llm, model_save_path)

100%|██████████| 929/929 [00:00<00:00, 105069.66it/s]


In [1]:
import torch
import os
import sys
sys.path.append('/home/lz/On-the-Fly_MoE_Inference')
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
from transformers import AutoTokenizer, MixtralForCausalLM
# from modeling_mixtral import MixtralForCausalLM
from convert_model import convert_mixtral_model
import json 

with open('../path.json', 'r') as f:
    path = json.load(f)
    model_name = path['mixtral']

with open('./device_map.json', 'r') as f:
    device_map = json.load(f)

llm = MixtralForCausalLM.from_pretrained(
    model_name,
    # device_map='auto',
    device_map=device_map,
    use_cache=True,
    torch_dtype=torch.float16,
    # attn_implementation="flash_attention_2"
) 
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
# convert_mixtral_model(llm, start_num=-1, end_num=32, gamma=0.2,)
# for name, param in llm.named_parameters():
#     print(name)

/home/lz/miniconda3/envs/hqq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 19/19 [00:22<00:00,  1.19s/it]


hqq.core.quantize.py L398: 修改self.offload_meta = True
<!-- (
            self.quant_config.pop("offload_meta")
            if (self.quant_config is not None)
            else None
        ) -->

In [2]:
#Quantize
from hqq.core.quantize import *
q4_config    = BaseQuantizeConfig(nbits=8, group_size=64, offload_meta=True) 
q3_config    = BaseQuantizeConfig(nbits=2, group_size=32)

quant_config = {
  'block_sparse_moe.experts.w3'  :q3_config,
}
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.quantize_model(llm, quant_config=quant_config, compute_dtype=torch.float16, device=device_map)
                              

  0%|          | 0/354 [00:00<?, ?it/s]

100%|██████████| 929/929 [00:14<00:00, 62.21it/s]


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=4096, out_features=14336, bias=False)
              (w2): Linear(in_features=14336, out_features=4096, bias=False)
              (w3): HQQLinear(in_features=4096, out_features=14336, bias=False)
 

In [6]:
llm.model.norm.weight.device

device(type='cuda', index=2)

In [7]:
AutoHQQHFModel.save_quantized(llm, './saved')

In [3]:
from hqq.core.quantize import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"
q4_config    = BaseQuantizeConfig(nbits=8, group_size=64, offload_meta=True) 
q3_config    = BaseQuantizeConfig(nbits=2, group_size=32)

quant_config = {
  'block_sparse_moe.experts.w3'  :q3_config,
}
from hqq.models.hf.base import AutoHQQHFModel
AutoHQQHFModel.load_weights('./saved', map_location='cuda')

{'model.embed_tokens': {'weight': tensor([[-0.0000e+00,  0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [-1.4954e-02,  2.5940e-04, -1.6113e-02,  ...,  2.2531e-05,
           -1.6846e-02,  2.1973e-03],
          [-5.3883e-05,  3.1948e-05,  6.3777e-06,  ...,  4.0293e-05,
            1.1623e-05,  1.1325e-06],
          ...,
          [ 8.7891e-03, -1.3000e-02,  5.2795e-03,  ...,  5.5237e-03,
            3.7079e-03,  3.7231e-03],
          [ 1.9073e-03,  1.8677e-02, -4.9133e-03,  ...,  1.3245e-02,
            4.6997e-03,  5.6152e-03],
          [ 3.6926e-03,  1.6235e-02,  1.8597e-04,  ..., -7.7820e-03,
           -9.5215e-03, -2.4292e-02]], device='cuda:0', dtype=torch.float16)},
 'model.layers.0.self_attn.q_proj': {'weight': tensor([[-2.5153e-05, -2.9449e-03,  5.1880e-04,  ...,  7.1106e-03,
            1.4126e-05,  5.4550e-04],
          [ 1.2159e-04,  2.2583e-03, -1.4496e-04,  ..., -6.9885e-03,
           -2.2221e-04, -5.9509e-04],
          [

### C4

In [3]:
from datasets import load_dataset
def preprocess_data(batch):
    # 使用 tokenizer 将文本数据转换为模型输入
    inputs = tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs["labels"] = inputs.input_ids.clone()
    return inputs

# 定义一个函数来选择特征并丢弃不需要的
def select_features(example):
    return {
        'input_ids': example['input_ids'],
        'attention_mask': example['attention_mask'],
        'labels': example['labels']
    }

tokenizer.pad_token = tokenizer.eos_token
# # 加载 C4 数据集的验证集
with open('../path.json', 'r') as file:
    paths = json.load(file)
    c4_path = paths.get('c4', '')
c4 = load_dataset(c4_path)
# 对数据集进行预处理
c4_dataset = c4.map(preprocess_data, batched=True)
# c4_dataset = c4_dataset.map(select_features, batched=True)
c4_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# c4_dataset
top_four_thousand_data = c4_dataset['validation'].select(range(10))

import numpy as np

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)

from torch.utils.data import DataLoader
from tqdm import tqdm
set_seed(42)

# 定义数据加载器
batch_size = 4
# dataloader = DataLoader(c4_dataset['validation'], batch_size=batch_size)
dataloader = DataLoader(top_four_thousand_data, batch_size=batch_size)

# 计算评估损失
total_loss = 0.0
num_batches = 0

for batch in tqdm(dataloader):
    input_ids = batch['input_ids'].to('cuda')
    attention_mask = batch['attention_mask'].to('cuda')
    labels = batch['labels'].to('cuda')
    
    # 禁用梯度计算
    with torch.no_grad():
        outputs = llm(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        num_batches += 1
        if num_batches % 100 == 0:
            print(f"[{num_batches}], Eval Loss: {total_loss / (num_batches)}")

# 计算平均损失
eval_loss = total_loss / num_batches
print(f"Eval Loss: {eval_loss}")

100%|██████████| 3/3 [00:03<00:00,  1.05s/it]

Eval Loss: 3.0374682744344077
